# Table and graphs for the supplementary information

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import os
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from matplotlib.ticker import FuncFormatter
from matplotlib.lines import Line2D
from utils import *
import matplotlib
matplotlib.rcParams['figure.dpi'] = 70

## No-response scenario

In [ ]:
# Define colors to differentiate between visitors and residents
colors = ['#FF2C00', '#0C5DA5']

# Define district population on 2020
population = {'Ypenburg': 21643, 
              'Centrum': 26833, 
              'Leidschenveen': 20_896, # 44
              'Schildersbuurt': 31_669 # 29
              }

# Load district shapefile
districts = gpd.read_file('../data/processed/cbs/wijk_buurt_kaart/districts.json')

total_population = 545_630

scenario = 'baseline'

# List all files corresponding to experimental setup
file_paths = find_files(f'../results/{scenario}', 'infectedPersons.csv')

# Print number of files/experiments
print(f"Number of experiments: {len(file_paths)}")

In [ ]:
# Define number of runs to load
n_runs = 10

# Extract all experiment names
experiment_names = [os.path.basename(os.path.dirname(
    file_path)) for file_path in file_paths][:n_runs]

In [ ]:
# Store start and end dates to find the smallest and largest
start_end_dates = {}

print('Loading experiment results...')
for experiment_name, file_path in tqdm(zip(experiment_names[:n_runs], file_paths[:n_runs]), total=len(file_paths[:n_runs])):
    # Load experiment results
    results = pd.read_csv(file_path)

    # # Convert to GeoDataFrame with infection location as geometry
    # results = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(
    #     results['infectLocationLon'], results['infectLocationLat']))

    # Add date_time column
    results['date_time'] = results['Time(h)'].apply(lambda x: relativedelta(
        years=50, months=2) + datetime.datetime(*time.gmtime(x * 3600)[:6]))

    # Get the start and end dates of the experiment
    experiment_start = results['date_time'].min().date()
    experiment_end = results['date_time'].max().date()

    # Store start and end dates
    start_end_dates[experiment_name] = (
        experiment_start, experiment_end)
    
# Find the smallest and largest start and end dates
min_start = min(start for start, _ in start_end_dates.values())
max_end = max(end for _, end in start_end_dates.values())

print(f"Smallest start date: {min_start}")
print(f"Largest end date: {max_end}")

# Define complete dates
complete_dates = pd.date_range(start=min_start, end=max_end, freq='d')
complete_dates[:5]

In [ ]:
# Iterate over start and end dates and print the number of days in the simulation
for experiment_name, (start, end) in start_end_dates.items():
    print(f"{experiment_name}: {start} - {end}")
    print(f"Number of days in simulation: {(end - start).days}")

In [ ]:
# Store infections in a dictionary
all_infections = {}
infections_by_day = {}
district_infections_by_day = {}
resident_visitor_infections_by_day = {}
district_resident_visitor_infections_by_day = {}

n_runs = 10

experiment_names = [os.path.basename(os.path.dirname(
    file_path)) for file_path in file_paths][:n_runs]

# Process each file
print('Loading experiment results...')
for experiment_name, file_path in tqdm(zip(experiment_names[:n_runs], file_paths[:n_runs]), total=len(file_paths[:n_runs])): 
    # Load experiment results
    results = pd.read_csv(file_path)
    
    # Convert to GeoDataFrame with infection location as geometry
    results = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(results['infectLocationLon'], results['infectLocationLat']))
    
    # Add date_time column
    results['date_time'] = results['Time(h)'].apply(lambda x: relativedelta(years=50, months=2) + datetime.datetime(*time.gmtime(x * 3600)[:6]))   
    
    # Add residence and infection districts to the data
    results = results.pipe(assign_residence_district, districts)\
                     .pipe(assign_infection_district, districts)
    
    # Add dummy variable indicating infection
    results['infection'] = 1

    # Add infection_type column based on whether infection occurred in district of residence
    results['infection_type'] = 'Visitor'  # Default to visitor
    
    # If infection district matches residence district, mark as resident
    mask = results['infection_district_name'] == results['residence_district_name']
    results.loc[mask, 'infection_type'] = 'Resident'

    # 1. Store all results
    all_infections[experiment_name] = results

    # 2. CALCULATE INFECTIONS BY DAY
    results_grouped = results.groupby(results['date_time'].dt.date)[
        'infection'].sum()

    # Reindex to include all complete_dates, filling missing rows with 0 for 'infection' and keeping other columns as NaN
    results_grouped = results_grouped.reindex(
        complete_dates, fill_value=np.nan)

    # Fill missing 'infection' values with 0
    results_grouped = results_grouped.fillna(0)
    
    # Store infections by day
    infections_by_day[experiment_name] = results_grouped

    # 3. DIFFERENTIATE BETWEEN INFECTIONS OF RESIDENTS AND VISITORS
    results_grouped = results.groupby(['infection_type', results['date_time'].dt.date])['infection'].sum()

    # Unstack so that 'date' is the index and 'infection_type' is columns
    results_grouped = results_grouped.unstack(level=0)

    # Reindex on the date index (second index originally), not the first
    results_grouped = results_grouped.reindex(complete_dates, fill_value=np.nan)
    results_grouped = results_grouped.fillna(0)

    # Stack back with infection_type as the first index
    results_grouped = results_grouped.stack().swaplevel(0, 1).sort_index()
    resident_visitor_infections_by_day[experiment_name] = results_grouped

    # 4. CALCULATE INFECTIONS BY DAY FOR THE DISTRICTS OF INTEREST
    results_grouped = results.groupby(['infection_district_name', results['date_time'].dt.date])['infection'].sum()
    results_grouped = results_grouped.unstack(level=0)
    results_grouped = results_grouped.reindex(complete_dates, fill_value=np.nan)
    results_grouped = results_grouped.fillna(0)
    results_grouped = results_grouped.stack().swaplevel(0, 1).sort_index()
    district_infections_by_day[experiment_name] = results_grouped

    # 5. CALCULATE INFECTIONS BY TYPE AND DISTRICT
    results_grouped = results.groupby(['infection_district_name', 'infection_type', results['date_time'].dt.date])['infection'].sum()

    # Unstack so that 'date' is the index and ('infection_district_name', 'infection_type') are columns
    results_grouped = results_grouped.unstack(level=2)

    # Reindex to ensure all complete_dates are present for each (district, type) pair
    results_grouped = results_grouped.reindex(columns=complete_dates, fill_value=np.nan)

    # Fill missing infection values with 0
    results_grouped = results_grouped.fillna(0)

    # Stack back so that index is (infection_district_name, infection_type, date)
    results_grouped = results_grouped.stack() 
    district_resident_visitor_infections_by_day[experiment_name] = results_grouped

infections_by_day = pd.concat(infections_by_day.values(), axis=1)
infections_by_day.columns = experiment_names
resident_visitor_infections_by_day = pd.concat(resident_visitor_infections_by_day.values(), axis=1)
resident_visitor_infections_by_day.columns = experiment_names
district_infections_by_day = pd.concat(district_infections_by_day.values(), axis=1)
district_infections_by_day.columns = experiment_names
district_resident_visitor_infections_by_day = pd.concat(district_resident_visitor_infections_by_day.values(), axis=1)
district_resident_visitor_infections_by_day.columns = experiment_names

In [ ]:
# COMPUTE METRICS OF INTEREST
infections_by_day_mean = infections_by_day.mean(axis=1)

infections_by_day_q1 = infections_by_day.quantile(0.1, axis=1)
infections_by_day_q3 = infections_by_day.quantile(0.9, axis=1)

# PLOTTING
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(
    infections_by_day.index,
    infections_by_day_mean,
    color='red',
    linewidth=1,
    label='Mean'
)
ax.fill_between(
    infections_by_day.index,
    infections_by_day_q1,
    infections_by_day_q3,
    alpha=0.5,
    color='gray',
    label='P10–P90 across runs'
)


# FORMATTING
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))
ax.set_xlabel('Simulation time (date)')

ax.set_ylabel('New infections per day (#)')

plt.xticks(rotation=45)
sns.despine()
plt.tight_layout()

legend_elements = [
    Line2D([0], [0], color='gray', alpha=0.5,
           linewidth=8, label='P10–P90 across runs'),
    Line2D([0], [0], color='red', label='Mean')
]

ax.legend(handles=legend_elements, loc='upper right', frameon=False)

# SAVING
for fmt in ['png', 'pdf', 'svg', 'eps']:
    plt.savefig(
        f'../figures/figs1a.{fmt}', dpi=300, bbox_inches='tight')

In [ ]:
infections_by_day.sum()

In [ ]:
totals = infections_by_day.sum(axis=0)

# Get main stats representing the diff across sim runs
infections_stats = pd.DataFrame({
    'Mean': totals.mean(),
    'Median': totals.median(),
    'Min': totals.min(),
    'Max': totals.max(),
    'Std': totals.std(ddof=1),
    'P10': totals.quantile(0.1),
    'P90': totals.quantile(0.9),
    'CV%': totals.std(ddof=1) / totals.mean() * 100
}, index=['No Response'])
infections_stats

In [ ]:
# COMPUTE METRICS OF INTEREST
y = infections_by_day.mean(axis=1) / total_population * 100
y_q1 = infections_by_day.quantile(0.1, axis=1) / total_population * 100
y_q3 = infections_by_day.quantile(0.9, axis=1) / total_population * 100

y_label = '% of population newly infected per day'
x_label = 'Simulation time (date)'

# PLOTTING
fig, ax = plt.subplots(figsize=(6, 4))

y.plot(ax=ax, legend=False, color='blue', linewidth=1)
ax.fill_between(y.index, y_q1, y_q3, alpha=0.5, color='gray')

# FORMATTING
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
plt.xticks(rotation=45)
ax.set_xlabel(x_label)
# ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))

ax.set_ylabel(y_label)

legend_elements = [
    Line2D([0], [0], color='gray', alpha=0.5,
           linewidth=8, label='P10–P90 across runs'),
    Line2D([0], [0], color='blue', label=f'Mean')
]
ax.legend(handles=legend_elements, loc='upper right', frameon=False)

sns.despine()

fig.tight_layout()

# SAVING
formats = ['png', 'pdf', 'svg', 'eps']
for format in formats:
    plt.savefig(
        f'../figures/figs1b.{format}', dpi=300, bbox_inches='tight')

In [ ]:
# 1) Cross-run variation of TOTAL infections (per 100 people)
# infections_by_day: index=time, columns=runs; values = new infections per day
# one scalar per run
totals = infections_by_day.sum(axis=0)
# % of population ever infected
pct_totals = (totals / total_population) * 100

infections_stats = pd.DataFrame({
    'Mean': [pct_totals.mean()],
    'Median': [pct_totals.median()],
    'Min': [pct_totals.min()],
    'P10': [pct_totals.quantile(0.10)],
    'P90': [pct_totals.quantile(0.90)],
    'Max': [pct_totals.max()],
    'Std': [pct_totals.std(ddof=1)],
    # coefficient of variation
    'CV%': [pct_totals.std(ddof=1) / pct_totals.mean() * 100],
}, index=['No Response'])
infections_stats

## Hard-lockdown scenario

In [ ]:
# Define colors to differentiate between visitors and residents
colors = ['#FF2C00', '#0C5DA5']

# Define district population on 2020
population = {'Ypenburg': 21643, 
              'Centrum': 26833, 
              'Leidschenveen': 20_896, # 44
              'Schildersbuurt': 31_669 # 29
              }

# Load district shapefile
districts = gpd.read_file('../data/processed/cbs/wijk_buurt_kaart/districts.json')

total_population = 545_630

scenario = 'lockdown'

# List all files corresponding to experimental setup
file_paths = find_files(f'../results/{scenario}', 'infectedPersons.csv')

# Print number of files/experiments
print(f"Number of experiments: {len(file_paths)}")

In [ ]:
# Define number of runs to load
n_runs = 10

# Extract all experiment names
experiment_names = [os.path.basename(os.path.dirname(
    file_path)) for file_path in file_paths][:n_runs]

In [ ]:
# Store start and end dates to find the smallest and largest
start_end_dates = {}

print('Loading experiment results...')
for experiment_name, file_path in tqdm(zip(experiment_names[:n_runs], file_paths[:n_runs]), total=len(file_paths[:n_runs])):
    # Load experiment results
    results = pd.read_csv(file_path)

    # # Convert to GeoDataFrame with infection location as geometry
    # results = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(
    #     results['infectLocationLon'], results['infectLocationLat']))

    # Add date_time column
    results['date_time'] = results['Time(h)'].apply(lambda x: relativedelta(
        years=50, months=2) + datetime.datetime(*time.gmtime(x * 3600)[:6]))

    # Get the start and end dates of the experiment
    experiment_start = results['date_time'].min().date()
    experiment_end = results['date_time'].max().date()

    # Store start and end dates
    start_end_dates[experiment_name] = (
        experiment_start, experiment_end)
    
# Find the smallest and largest start and end dates
min_start = min(start for start, _ in start_end_dates.values())
max_end = max(end for _, end in start_end_dates.values())

print(f"Smallest start date: {min_start}")
print(f"Largest end date: {max_end}")

# Define complete dates
complete_dates = pd.date_range(start=min_start, end=max_end, freq='d')
complete_dates[:5]

In [ ]:
# Iterate over start and end dates and print the number of days in the simulation
for experiment_name, (start, end) in start_end_dates.items():
    print(f"{experiment_name}: {start} - {end}")
    print(f"Number of days in simulation: {(end - start).days}")

In [ ]:
# Store infections in a dictionary
all_infections = {}
infections_by_day = {}
district_infections_by_day = {}
resident_visitor_infections_by_day = {}
district_resident_visitor_infections_by_day = {}

n_runs = 10

experiment_names = [os.path.basename(os.path.dirname(
    file_path)) for file_path in file_paths][:n_runs]

# Process each file
print('Loading experiment results...')
for experiment_name, file_path in tqdm(zip(experiment_names[:n_runs], file_paths[:n_runs]), total=len(file_paths[:n_runs])): 
    # Load experiment results
    results = pd.read_csv(file_path)
    
    # Convert to GeoDataFrame with infection location as geometry
    results = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(results['infectLocationLon'], results['infectLocationLat']))
    
    # Add date_time column
    results['date_time'] = results['Time(h)'].apply(lambda x: relativedelta(years=50, months=2) + datetime.datetime(*time.gmtime(x * 3600)[:6]))   
    
    # Add residence and infection districts to the data
    results = results.pipe(assign_residence_district, districts)\
                     .pipe(assign_infection_district, districts)
    
    # Add dummy variable indicating infection
    results['infection'] = 1

    # Add infection_type column based on whether infection occurred in district of residence
    results['infection_type'] = 'Visitor'  # Default to visitor
    
    # If infection district matches residence district, mark as resident
    mask = results['infection_district_name'] == results['residence_district_name']
    results.loc[mask, 'infection_type'] = 'Resident'

    # 1. Store all results
    all_infections[experiment_name] = results

    # 2. CALCULATE INFECTIONS BY DAY
    results_grouped = results.groupby(results['date_time'].dt.date)[
        'infection'].sum()

    # Reindex to include all complete_dates, filling missing rows with 0 for 'infection' and keeping other columns as NaN
    results_grouped = results_grouped.reindex(
        complete_dates, fill_value=np.nan)

    # Fill missing 'infection' values with 0
    results_grouped = results_grouped.fillna(0)
    
    # Store infections by day
    infections_by_day[experiment_name] = results_grouped

    # 3. DIFFERENTIATE BETWEEN INFECTIONS OF RESIDENTS AND VISITORS
    results_grouped = results.groupby(['infection_type', results['date_time'].dt.date])['infection'].sum()

    # Unstack so that 'date' is the index and 'infection_type' is columns
    results_grouped = results_grouped.unstack(level=0)

    # Reindex on the date index (second index originally), not the first
    results_grouped = results_grouped.reindex(complete_dates, fill_value=np.nan)
    results_grouped = results_grouped.fillna(0)

    # Stack back with infection_type as the first index
    results_grouped = results_grouped.stack().swaplevel(0, 1).sort_index()
    resident_visitor_infections_by_day[experiment_name] = results_grouped

    # 4. CALCULATE INFECTIONS BY DAY FOR THE DISTRICTS OF INTEREST
    results_grouped = results.groupby(['infection_district_name', results['date_time'].dt.date])['infection'].sum()
    results_grouped = results_grouped.unstack(level=0)
    results_grouped = results_grouped.reindex(complete_dates, fill_value=np.nan)
    results_grouped = results_grouped.fillna(0)
    results_grouped = results_grouped.stack().swaplevel(0, 1).sort_index()
    district_infections_by_day[experiment_name] = results_grouped

    # 5. CALCULATE INFECTIONS BY TYPE AND DISTRICT
    results_grouped = results.groupby(['infection_district_name', 'infection_type', results['date_time'].dt.date])['infection'].sum()

    # Unstack so that 'date' is the index and ('infection_district_name', 'infection_type') are columns
    results_grouped = results_grouped.unstack(level=2)

    # Reindex to ensure all complete_dates are present for each (district, type) pair
    results_grouped = results_grouped.reindex(columns=complete_dates, fill_value=np.nan)

    # Fill missing infection values with 0
    results_grouped = results_grouped.fillna(0)

    # Stack back so that index is (infection_district_name, infection_type, date)
    results_grouped = results_grouped.stack() 
    district_resident_visitor_infections_by_day[experiment_name] = results_grouped

infections_by_day = pd.concat(infections_by_day.values(), axis=1)
infections_by_day.columns = experiment_names
resident_visitor_infections_by_day = pd.concat(resident_visitor_infections_by_day.values(), axis=1)
resident_visitor_infections_by_day.columns = experiment_names
district_infections_by_day = pd.concat(district_infections_by_day.values(), axis=1)
district_infections_by_day.columns = experiment_names
district_resident_visitor_infections_by_day = pd.concat(district_resident_visitor_infections_by_day.values(), axis=1)
district_resident_visitor_infections_by_day.columns = experiment_names

In [ ]:
# COMPUTE METRICS OF INTEREST
infections_by_day_mean = infections_by_day.mean(axis=1)

infections_by_day_q1 = infections_by_day.quantile(0.1, axis=1)
infections_by_day_q3 = infections_by_day.quantile(0.9, axis=1)

# PLOTTING
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(
    infections_by_day.index,
    infections_by_day_mean,
    color='red',
    linewidth=1,
    label='Mean'
)
ax.fill_between(
    infections_by_day.index,
    infections_by_day_q1,
    infections_by_day_q3,
    alpha=0.5,
    color='gray',
    label='P10–P90 across runs'
)

# Add horizontal line at day 20 (hard lockdown)
lockdown_date = infections_by_day.index[0] + pd.Timedelta(days=20)
ax.axvline(x=lockdown_date, color='black', linestyle='--', linewidth=1, alpha=0.7)
ax.text(lockdown_date, ax.get_ylim()[1] * 0.9, 'Hard lockdown', 
        rotation=90, verticalalignment='top', horizontalalignment='right',
        fontsize=9, alpha=0.8)

# FORMATTING
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))
ax.set_xlabel('Simulation time (date)')

ax.set_ylabel('New infections per day (#)')

plt.xticks(rotation=45)
sns.despine()
plt.tight_layout()

legend_elements = [
    Line2D([0], [0], color='gray', alpha=0.5,
           linewidth=8, label='P10–P90 across runs'),
    Line2D([0], [0], color='red', label='Mean'),
    Line2D([0], [0], color='black', linestyle='--', linewidth=1, alpha=0.7, label='Hard lockdown')
]

ax.legend(handles=legend_elements, loc='upper right', frameon=False)

# SAVING
for fmt in ['png', 'pdf', 'svg', 'eps']:
    plt.savefig(
        f'../figures/figs2a.{fmt}', dpi=300, bbox_inches='tight')

In [ ]:
totals = infections_by_day.sum(axis=0)

# Get main stats representing the diff across sim runs
infections_stats = pd.DataFrame({
    'Mean': totals.mean(),
    'Median': totals.median(),
    'Min': totals.min(),
    'Max': totals.max(),
    'Std': totals.std(ddof=1),
    'P10': totals.quantile(0.1),
    'P90': totals.quantile(0.9),
    'CV%': totals.std(ddof=1) / totals.mean() * 100
}, index=['Hard Lockdown'])
infections_stats

In [ ]:
# COMPUTE METRICS OF INTEREST
y = infections_by_day.mean(axis=1) / total_population * 100
y_q1 = infections_by_day.quantile(0.1, axis=1) / total_population * 100
y_q3 = infections_by_day.quantile(0.9, axis=1) / total_population * 100

y_label = '% of population newly infected per day'
x_label = 'Simulation time (date)'

# PLOTTING
fig, ax = plt.subplots(figsize=(6, 4))

y.plot(ax=ax, legend=False, color='blue', linewidth=1)
ax.fill_between(y.index, y_q1, y_q3, alpha=0.5, color='gray')

# Add horizontal line at day 20 (hard lockdown)
lockdown_date = y.index[0] + pd.Timedelta(days=20)
ax.axvline(x=lockdown_date, color='black', linestyle='--', linewidth=1, alpha=0.7)
ax.text(lockdown_date, ax.get_ylim()[1] * 0.9, 'Hard lockdown', 
        rotation=90, verticalalignment='top', horizontalalignment='right',
        fontsize=9, alpha=0.8)

# FORMATTING
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
plt.xticks(rotation=45)
ax.set_xlabel(x_label)
# ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))

ax.set_ylabel(y_label)

legend_elements = [
    Line2D([0], [0], color='gray', alpha=0.5,
           linewidth=8, label='P10–P90 across runs'),
    Line2D([0], [0], color='blue', label=f'Mean'),
    Line2D([0], [0], color='black', linestyle='--', linewidth=1, alpha=0.7, label='Hard lockdown')
]
ax.legend(handles=legend_elements, loc='upper right', frameon=False)

sns.despine()

fig.tight_layout()

# SAVING
formats = ['png', 'pdf', 'svg', 'eps']
for format in formats:
    plt.savefig(
        f'../figures/figs2b.{format}', dpi=300, bbox_inches='tight')

In [ ]:
totals = infections_by_day.sum(axis=0)
pct_totals = (totals / total_population) * 100

infections_stats = pd.DataFrame({
    'Mean': [pct_totals.mean()],
    'Median': [pct_totals.median()],
    'Min': [pct_totals.min()],
    'P10': [pct_totals.quantile(0.10)],
    'P90': [pct_totals.quantile(0.90)],
    'Max': [pct_totals.max()],
    'Std': [pct_totals.std(ddof=1)],
    'CV%': [pct_totals.std(ddof=1) / pct_totals.mean() * 100],
}, index=['Hard Lockdown'])
infections_stats